In [ ]:
import os
import pandas as pd
from llama_cloud_services import LlamaExtract
from dotenv import load_dotenv
from typing import Optional
from pydantic import ValidationError

from pydantic import BaseModel, Field
from typing import Optional, Type

from BS_Schema import make_StatementOfFinancialPosition_model

import BS_Schema
from importlib import reload

BS_Schema = reload(BS_Schema_250826)

In [2]:
## Import Class
SFP = make_StatementOfFinancialPosition_model(2024)
# 4.2) Inspect its name and fields
print(SFP.__name__)           # → "StatementOfFinancialPosition_2024"
print(list(SFP.__fields__.keys()))

StatementOfFinancialPosition_2024
['cash_and_short_term_investments_unrestricted_and_restricted', 'accounts_receivable', 'pledges_receivable', 'government_grants_and_other_receivables', 'loans_receivable', 'receivables_leftover', 'all_receivables', 'accumulated_depreciation_bs', 'accumulated_amortization_bs', 'rou_assets_finance_lease_bs', 'net_fixed_assets_raw', 'rou_assets_operating_lease_bs', 'long_term_investments', 'cash_surrender_value_life_insurance', 'deferred_revenue_raw', 'student_tuition_and_deposits', 'student_credit_balances_and_deposits', 'finance_lease_liability_bs', 'long_term_debt_labeled', 'bonds_payable', 'other_long_term_debt_obligations', 'operating_lease_liability_bs', 'pension_liability', 'swap_obligation_fmv', 'opeb_liability', 'pension_and_opeb_liability', 'other_liabilities', 'accumulated_depreciation_notes', 'accumulated_amortization_notes', 'rou_assets_finance_lease_notes', 'rou_assets_operating_lease_notes', 'finance_lease_liability_notes', 'operating_lease

C:\Users\wilso\AppData\Local\Temp\ipykernel_13112\528350060.py:5: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(list(SFP.__fields__.keys()))


In [3]:
PDF_ROOT = "private_universities/university_pdfs/"
OUTPUT_ROOT = "output_Balance_Sheet_DS"
os.makedirs(OUTPUT_ROOT, exist_ok=True)
AGENT_ID = "bcb15a18-67ac-4772-9965-2654ecaff88c" #Different based on your LLamaCloud account
load_dotenv() #make sure the API key is in the .env file
Year = 2024
extractor = LlamaExtract(api_key='llx-jlyOr0ZQwOzn0BPYgzggmo6mFShTLTsxdrXGviOaBf6IDnHG',project_id = '8c10e62e-3810-4193-915d-d2d11105826d')# LlamaExtract(project_id = '8c10e62e-3810-4193-915d-d2d11105826d')

# #uncomment the below line if you are creating the agent for the first time
# agent = extractor.create_agent(name = "balance-sheet-parser-v1", data_schema=SFP)

agent = extractor.get_agent(id = AGENT_ID)

# uncomment the following lines if you updated the schema
agent.data_schema = SFP
agent.save()
# agent = extractor.get_agent(id = AGENT_ID)

In [4]:
#check data schema
agent.data_schema

{'additionalProperties': False,
 'properties': {'cash_and_short_term_investments_unrestricted_and_restricted': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': "Extract **only** from the page or section with a heading such as 'Consolidated Balance Sheet' 2024 or 'Statement of Financial Position' 2024. Do not extract this information from any page or section with headings such as 'Cash Flow Statement' 2024, 'Statement of Cash Flows' 2024, 'Statements of Activities' 2024, 'Statement of Functional Expenses' 2024, 'Notes to Financial Statements' 2024, 'Notes to Consolidated Financial Statements' 2024, or 'Footnotes'.Sum the amounts from asset line items in the 'Consolidated Balance Sheet' 2024 or 'Statement of Financial Position' 2024 that include keywords like 'Cash and cash equivalents' or 'Short-term investments'. Include both unrestricted and restricted components."},
  'accounts_receivable': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
   'description': "

In [5]:
def process_school(school_name, school_dir):
    combined   = {}
    first_keys = None

    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        outfile = os.path.join(OUTPUT_ROOT, f"{school_name}.xlsx")
        df.to_excel(outfile)
        print(f"Saved output to {outfile}")
    else:
        print(f"No PDF data found for {school_name}")

In [6]:
results = {}

for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue

    # 1) Gather all PDFs in the directory
    pdf_files = [
        f for f in sorted(os.listdir(school_dir))
        if f.lower().endswith(".pdf")
    ]

    # 2) Decide which PDFs to read:
    if len(pdf_files) == 1:
        to_read = pdf_files
    else:
        financial_only = [f for f in pdf_files if "financial" in f.lower()]
        to_read = financial_only if financial_only else pdf_files

    combined   = {}
    first_keys = None

    for fname in to_read:
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        # store this school's combined dict in results
        results[school] = combined
    else:
        print(f"No data for {school}.")


# Build one DataFrame: rows=schools, columns=metrics
df_all = pd.DataFrame.from_dict(results, orient="index")




Extracting data from CORNELL_UNIVERSITY/2024_Audited_Financial_Statements_for_the_year_ended_06_30_2024__788_KB_.pdf


Extracting files: 100%|██████████| 1/1 [01:55<00:00, 115.06s/it]


Extracting data from CULINARY_INSTITUTE_OF_AMERICA_THE/2024_Audited_Financial_Statements_for_the_year_ended_05_31_2024__277_KB_.pdf


Extracting files: 100%|██████████| 1/1 [02:33<00:00, 153.78s/it]


Extracting data from GANNON_UNIVERSITY/Audited_Financial_Statements_for_the_year_ended_06_30_2024__786_KB_.pdf


Extracting files: 100%|██████████| 1/1 [01:10<00:00, 70.85s/it]


Extracting data from LEWIS_UNIVERSITY/Audited_Financial_Statements_for_the_year_ended_06_30_2024__430_KB_.pdf


Extracting files: 100%|██████████| 1/1 [01:32<00:00, 92.27s/it]


Extracting data from MOLLOY_COLLEGE/Financial_Operating_Filing_for_the_year_ended_06_30_2024_Document1__304_KB_.pdf


Extracting files: 100%|██████████| 1/1 [01:55<00:00, 115.27s/it]


In [7]:
from typing import List

def add_plug_accounts(df: pd.DataFrame) -> pd.DataFrame:
    """
    Given a DataFrame where each row is a university and columns include:
      - 'total_assets', 'other_assets', [various asset components…]
      - 'total_liabilities', 'other_liabilities', [various liability components…]
    this will compute two new columns:
    
      * other_assets_plug      = total_assets - sum(all other asset components)
      * other_liabilities_plug  = total_liabilities - sum(all other liability components)
    
    so that:
      sum(assets components) + other_assets_plug == total_assets
      sum(liability components) + other_liabilities_plug == total_liabilities
    
    Returns the DataFrame with these two new columns.
    """

    def compute_combined_fields(df: pd.DataFrame) -> pd.DataFrame:
        """
        Compute combined fields by summing related *_bs and *_notes columns
        into *_sum columns. Missing values are treated as 0.
        """
        pairs_to_sum = {
            "rou_assets_finance_lease": ["rou_assets_finance_lease_bs", "rou_assets_finance_lease_notes"],
            "rou_assets_operating_lease": ["rou_assets_operating_lease_bs", "rou_assets_operating_lease_notes"],
            # "finance_lease_liability": ["finance_lease_liability_bs", "finance_lease_liability_notes"],
            # "operating_lease_liability": ["operating_lease_liability_bs", "operating_lease_liability_notes"]
            # "rou_assets_finance_lease": ["rou_assets_finance_lease_bs"],
            # "rou_assets_operating_lease": ["rou_assets_operating_lease_bs"],
            "finance_lease_liability": ["finance_lease_liability_bs"],
            "operating_lease_liability": ["operating_lease_liability_bs"]
        }

        # for new_col, (col1, col2) in pairs_to_sum.items():
        #     df[new_col] = df.get(col1, 0).fillna(0) + df.get(col2, 0).fillna(0)

        # for new_col, cols_to_sum in pairs_to_sum.items():
        #     df[new_col] = df[cols_to_sum].fillna(0).sum(axis=1)

        for new_col, cols_to_sum in pairs_to_sum.items():
            if len(cols_to_sum) == 1:
                # Only one column → just copy it over
                df[new_col] = df[cols_to_sum[0]]

            else:
                col1, col2 = cols_to_sum
                df[new_col] = df.apply(
                    lambda row: (
                        row[col1] if pd.isna(row[col2]) else
                        row[col2] if pd.isna(row[col1]) else
                        row[col1] if row[col1] == row[col2] else
                        row[col1] + row[col2]
                    ),
                    axis=1
                )

        return df

    df = compute_combined_fields(df)

    net_receivables_components = [
        "accounts_receivable",
        # "student_accounts_receivable",
        "pledges_receivable",
        "government_grants_and_other_receivables",
        # "student_loans_receivable",
        "loans_receivable",
        # "grants_and_other_accounts_receivable",
        # "long_term_loans_to_students",
        "receivables_leftover"
    ]

    # Sum into final `net_receivables`
    df["net_receivables"] = df[net_receivables_components].fillna(0).sum(axis=1)

    df["current_portion_finance_lease"] = (
        df.get("current_portion_finance_lease_raw", 0).fillna(0)
        + df.get("current_portion_finance_lease_notes_due_next_year", 0).fillna(0)
    )

    # df["current_portion_operating_lease"] = (
    #     df.get("current_portion_operating_lease_raw", 0).fillna(0)
    #     + df.get("current_portion_operating_lease_notes_due_next_year", 0).fillna(0)
    # )

    df["current_portion_operating_lease"] = (
        df["current_portion_operating_lease_raw"].combine_first(
            df["current_portion_operating_lease_notes_due_next_year"]
        )
    )

    df["accumulated_depreciation"] = (
        df.get("accumulated_depreciation_bs", 0).fillna(0)
        + df.get("accumulated_amortization_bs", 0).fillna(0)
        + df.get("accumulated_depreciation_notes", 0).fillna(0)
        + df.get("accumulated_amortization_notes", 0).fillna(0)
    )

    df["long_term_investments_unrestricted_and_restricted"] = (
        df.get("long_term_investments", 0).fillna(0)
        + df.get("cash_surrender_value_life_insurance", 0).fillna(0)
    )

    df["deferred_revenue"] = (
        df.get("deferred_revenue_raw", 0).fillna(0)
        + df.get("student_tuition_and_deposits", 0).fillna(0)
        + df.get("student_credit_balances_and_deposits", 0).fillna(0)
    )

    df["long_term_debt_raw"] = (
        df.get("long_term_debt_labeled", 0).fillna(0)
        + df.get("bonds_payable", 0).fillna(0)
        + df.get("other_long_term_debt_obligations", 0).fillna(0)
    )

    # ADDED for modifying net_fixed_assets 
    df["net_fixed_assets"] = df["net_fixed_assets_raw"].fillna(0) + df["rou_assets_finance_lease"].fillna(0)
    # df["current_portion_long_term_debt"] = df.get("current_portion_long_term_debt_raw", 0).fillna(0) + df.get("current_portion_long_term_debt_notes_due_next_year", 0).fillna(0) + df["current_portion_finance_lease"].fillna(0)
    df["current_portion_long_term_debt"] = df.get("current_portion_long_term_debt_notes_due_next_year", 0).fillna(0) # + df["current_portion_finance_lease"].fillna(0)
    df["long_term_debt"] = df["long_term_debt_raw"].fillna(0) + df["finance_lease_liability"].fillna(0)

    # define the “known” component column names
    asset_components = [
        "cash_and_short_term_investments_unrestricted_and_restricted",
        "net_receivables",
        "net_fixed_assets",
        "long_term_investments_unrestricted_and_restricted",
        # "rou_assets_finance_lease",
        "rou_assets_operating_lease"
    ]
    liability_components = [
        "short_term_debt",
        "current_portion_finance_lease",
        "current_portion_long_term_debt",
        "current_portion_operating_lease",
        "accounts_payable",
        "deferred_revenue",
        "long_term_debt",
        # ADDING rename
        # "long_term_finance_lease",
        "finance_lease_liability",
        # ADDING rename
        # "long_term_operating_lease",
        "operating_lease_liability",
        "swap_obligation_fmv",
        "pension_and_opeb_liability",
        "pension_liability",
        "opeb_liability"
    ]
    
    # pick only those actually in the DataFrame
    assets_cols = [c for c in asset_components if c in df.columns]
    liabs_cols  = [c for c in liability_components if c in df.columns]
    
    # compute the sum of components (skip NaNs)
    df["sum_asset_components"]     = df[assets_cols].sum(axis=1, skipna=True)
    df["sum_liability_components"] = df[liabs_cols].sum(axis=1, skipna=True)
    
    # now compute the plugs
    df["other_assets_plug"]     = df["total_assets"] - df["sum_asset_components"]
    df["other_liabilities_plug"] = df["total_liabilities"] - df["sum_liability_components"]
    
    # ADDED for net assets
    df["net_assets_calculated"] = df["total_assets"] - df["total_liabilities"]

    # (optional) drop the intermediate sum columns if you like
    # df.drop(columns=["sum_asset_components", "sum_liability_components"], inplace=True)
    
    return df


In [8]:
# add plug accounts and make adjustment for expendable assets
df_allv1 = add_plug_accounts(df_all)
df_allv1['expendable_net_assets_with_donor_restrictions_calculated']=df_allv1['net_assets_with_donor_restrictions'] - df_allv1['perpetual_net_assets_with_donor_restrictions']
df_allv1.rename(columns={'expendable_net_assets_with_donor_restrictions':'expendable_net_assets_with_donor_restrictions_extracted'},inplace=True)

In [9]:
desired_order = [
    "year", "cash_and_short_term_investments_unrestricted_and_restricted", "net_receivables",
    "accumulated_depreciation", "rou_assets_finance_lease", "net_fixed_assets",
    "rou_assets_operating_lease", "long_term_investments_unrestricted_and_restricted",
    "other_assets_plug", "other_assets", "total_assets", "current_portion_finance_lease",
    "current_portion_long_term_debt", "current_portion_operating_lease", "short_term_debt",
    "accounts_payable", "deferred_revenue", "finance_lease_liability", "long_term_debt",
    "operating_lease_liability", "swap_obligation_fmv", "pension_liability", "opeb_liability",
    "pension_and_opeb_liability", "other_liabilities_plug", "other_liabilities", "total_liabilities",
    "net_assets", "net_assets_without_donor_restrictions", "expendable_net_assets_with_donor_restrictions_calculated",
    "perpetual_net_assets_with_donor_restrictions", "net_assets_with_donor_restrictions",
    "noncontrolling_interest", "expendable_net_assets_with_donor_restrictions_extracted",
    "total_net_assets", "total_liabilities_and_net_assets",
    "units_multiplier", "sum_asset_components", "sum_liability_components",
    "net_assets_calculated", "contribution_type",

    "accumulated_depreciation_bs", "accumulated_amortization_bs",
    "accumulated_depreciation_notes", "accumulated_amortization_notes",
    "rou_assets_finance_lease_bs", "rou_assets_finance_lease_notes",
    "rou_assets_operating_lease_bs", "rou_assets_operating_lease_notes",
    "finance_lease_liability_bs", "finance_lease_liability_notes",
    "operating_lease_liability_bs", "operating_lease_liability_notes",
    "current_portion_finance_lease_raw", "current_portion_finance_lease_notes_due_next_year",
    "current_portion_operating_lease_raw", "current_portion_operating_lease_notes_due_next_year",
    "current_portion_long_term_debt_raw", "current_portion_long_term_debt_notes_due_next_year",
    "long_term_investments", "cash_surrender_value_life_insurance",
    "deferred_revenue_raw", "student_tuition_and_deposits", "student_credit_balances_and_deposits",
    "long_term_debt_labeled", "bonds_payable", "other_long_term_debt_obligations"
]

remaining_columns = [col for col in df_allv1.columns if col not in desired_order]
full_column_order = desired_order + remaining_columns

df_reordered = df_allv1[full_column_order]

# df_reordered = df_allv1.reindex(columns=desired_order)

In [10]:
OUTPUT_FILE = "all_private_universities_balance_sheet.xlsx"

# save output
df_reordered.to_excel(OUTPUT_FILE)